In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian
# YOUR CODE HERE (OPTION) 
# Nếu cần các thư viện khác thì bạn có thể import ở đây

In [9]:
r = requests.get('https://api-v2.soundcloud.com/tracks/1126948735?client_id=ZyeUddavtQufdSASlMNmlO6oGkXi347q')
data = json.loads(r.text)
data

{'artwork_url': 'https://i1.sndcdn.com/artworks-WW7HaRJtQy5EV6wm-AvKzlA-large.jpg',
 'caption': None,
 'commentable': True,
 'comment_count': 49,
 'created_at': '2021-09-18T14:27:16Z',
 'description': '',
 'downloadable': False,
 'download_count': 0,
 'duration': 207543,
 'full_duration': 207543,
 'embeddable_by': 'all',
 'genre': 'Pop',
 'has_downloads_left': False,
 'id': 1126948735,
 'kind': 'track',
 'label_name': None,
 'last_modified': '2021-10-06T23:22:06Z',
 'license': 'all-rights-reserved',
 'likes_count': 13185,
 'permalink': 'diu-dang-em-den-lofi-erik-x-quanvrox',
 'permalink_url': 'https://soundcloud.com/h-ng-v-787579738/diu-dang-em-den-lofi-erik-x-quanvrox',
 'playback_count': 899886,
 'public': True,
 'publisher_metadata': {'id': 1126948735,
  'urn': 'soundcloud:tracks:1126948735',
  'contains_music': True},
 'purchase_title': None,
 'purchase_url': None,
 'release_date': None,
 'reposts_count': 242,
 'secret_token': None,
 'sharing': 'public',
 'state': 'finished',
 'str

In [3]:
#b3
def collect_repositories(start_date, end_date, keyword='data science', per_page=100):
    repositories = []
    # YOUR CODE HERE
    #ex: https://api.github.com/search/repositories?q=data%20science%20created:2017-01-01..2017-01-15&per_page=100
    url = f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}&per_page={per_page}'
    page = 0

    while(1):   
        #preconditions: check if the page is 'valid' to continue
        while(1):
            r = requests.get(url)
            response = json.loads(r.content)

            if r.ok == False or response['incomplete_results'] == True:
                time.sleep(sleep_time)
            else:
                break
        
        # process each item (= each dict) in items 
        items = response['items']
        count_item = 0
        
        for item in items:
            if count_item <= per_page:   
                repositories.append(item)
                count_item += 1
            else:
                break
        
        # go to next page
        # example: '<https://api.github.com/search/repositories?q=data+science+created%3A2017-01-01..2017-01-31&per_page=100&page=2>; rel="next", <https://api.github.com/search/repositories?q=data+science+created%3A2017-01-01..2017-01-31&per_page=100&page=10>; rel="last"'
        if per_page >= response['total_count']: # this is the first & also the last page
            break
        else:
            list_links = r.headers['Link']
            match = re.search('rel="next"', list_links)
            
            if not match: 
                break
            else:
                end_boundary = match.start()-3
                if page == 0: start_boundary = 1
                else: start_boundary = re.search('rel="prev"', list_links).end() + 3
                url = list_links[start_boundary:end_boundary]
            page += 1
        
    return repositories 

In [4]:
def collect_all_repositories(start_date, end_date, step, keyword='data science', per_page=100):
    all_repositories = []
    # YOUR CODE HERE
    start_date = datetime.strptime(start_date,"%Y-%m-%d")
    end_date = datetime.strptime(end_date,"%Y-%m-%d")
    tdelta = timedelta(days = step)
    last_day = False
    
    while True:
     if last_day:
      break
    
     date = start_date + tdelta
        
     if(date > end_date):
      date = end_date
      last_day = True
        
     res = collect_repositories(datetime.strftime(start_date,"%Y-%m-%d"), datetime.strftime(date,"%Y-%m-%d"), keyword, per_page)
     all_repositories.extend(res)
     start_date = start_date + tdelta + timedelta(days=1)
        
    return all_repositories

In [5]:
# TEST
start_date = '2017-01-01'
end_date = '2017-01-31'
step = 10
keyword = 'data science'
per_page = 100
all_repositories = collect_all_repositories(start_date, end_date, step, keyword, per_page)
print(len(all_repositories))

url = f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}'
while True:
    json_str = requests.get(url).text
    if '"incomplete_results":false' in json_str:
        i = json_str.find(':')
        j = json_str.find(',')
        total_count = int(json_str[i+1:j])
        break

assert len(all_repositories) == total_count 


1545
